In [ ]:
# This file contaiins notebook code for client1

In [2]:
# importing all the libraries and depenedencies

!pip install -qqq pandas
!pip install -qqq scikit-learn
!pip install -qqq flwr
!pip install -qqq tensorflow

import flwr as fl
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns
import math
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Conv1D
from tensorflow.keras.layers import MaxPool1D

# disabling warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
M_PI=3.1416
def compute_roll_yaw_pitch(x,y,z):
  #Acceleration around X
  acc_x_accl=[]

  #Acceleration around Y
  acc_y_accl=[]

  #Acceleration arounf Z
  acc_z_accl=[]


  for (x_mean,y_mean,z_mean) in zip(x,y,z):
    acc_x_accl.append(float("{:.2f}".format(x_mean*3.9)))
    acc_y_accl.append(float("{:.2f}".format(y_mean*3.9)))
    acc_z_accl.append(float("{:.2f}".format(z_mean*3.9)))


  acc_pitch=[]
  acc_roll=[]
  acc_yaw=[]

  for (acc_x,acc_y,acc_z) in zip(acc_x_accl,acc_y_accl,acc_z_accl):
    if acc_y==0 and acc_z==0:
      value_pitch=-0.1
    else:
      value_pitch=180 * math.atan (acc_x/math.sqrt(acc_y*acc_y + acc_z*acc_z))/M_PI
    if acc_x ==0 and acc_z==0:
      value_roll=-0.1
      value_yaw=-0.1
    else:
      value_roll = 180 * math.atan (acc_y/math.sqrt(acc_x*acc_x + acc_z*acc_z))/M_PI
      value_yaw = 180 * math.atan (acc_z/math.sqrt(acc_x*acc_x + acc_z*acc_z))/M_PI
    value_pitch=float("{:.2f}".format(value_pitch))
    value_roll=float("{:.2f}".format(value_roll))
    value_yaw=float("{:.2f}".format(value_yaw))
    acc_pitch.append(value_pitch)
    acc_roll.append(value_roll)
    acc_yaw.append(value_yaw)
  return acc_pitch,acc_roll,acc_yaw

In [4]:
#Sliding Window to null values
def fill_null(data):
  for col in data.columns:
    null_indexes=data[data[col].isnull()].index.tolist()
    #print("For ",col)
    for ind in null_indexes:
      #print(" Got null value at ",ind)
      values=data.loc[ind-6:ind-1,col]
      #print(" Last 5 values ",values)
      mean_val=values.mean()
      data.loc[ind,col]=mean_val
  return data

In [5]:
# loading the data

def load_data():
    user_one = "/Users/carbon/Desktop/Federated Learning/extrasensory_dataset/0A986513-7828-4D53-AA1F-E02D6DF9561B.features_labels.csv.gz"
    # user_one="C:\\pdata\\extrasensory_dataset\\00EABED2-271D-49D8-B599-1D4A09240601.features_labels.csv.gz"
    df_one=pd.read_csv(user_one)
    #accelerometer
    df_acc=df_one.iloc[:,1:27]
    df_acc=fill_null(df_acc)
    #gyroscope
    df_gyro=df_one.iloc[:,27:53]
    df_gyro=fill_null(df_gyro)
    #magnometer
    df_magnet=df_one.iloc[:,53:84]
    df_magnet=fill_null(df_magnet)
    # watch accelerometer
    #df_watch_acc=df_one.iloc[:,84:130]
    # location
    #df_location=df_one.iloc[:,139:156]

    # For accelerometer
    #mean values
    acc_mean_x=df_acc['raw_acc:3d:mean_x']
    acc_mean_y=df_acc['raw_acc:3d:mean_y']
    acc_mean_z=df_acc['raw_acc:3d:mean_z']

    acc_mean_x=acc_mean_x.replace({0:0.001})

    #standard deviations
    #acc_std_x=df_acc['raw_acc:3d:std_x']
    #acc_std_y=df_acc['raw_acc:3d:std_y']
    #acc_std_z=df_acc['raw_acc:3d:std_z']

    (pitch,roll,yaw)=compute_roll_yaw_pitch(acc_mean_x,acc_mean_y,acc_mean_z)
    df_one['acc_pitch']=pitch
    df_one['acc_roll']=roll
    df_one['acc_yaw']=yaw

    #for gyroscope
    gyro_mean_x=df_gyro['proc_gyro:3d:mean_x']
    gyro_mean_y=df_gyro['proc_gyro:3d:mean_y']
    gyro_mean_z=df_gyro['proc_gyro:3d:mean_z']

    (pitch,roll,yaw)=compute_roll_yaw_pitch(gyro_mean_x,gyro_mean_y,gyro_mean_z)

    df_one['gyro_pitch']=pitch
    df_one['gyro_roll']=roll
    df_one['gyro_yaw']=yaw

    # For magnetometer
    magno_mean_x=df_magnet['raw_magnet:3d:mean_x']
    magno_mean_y=df_magnet['raw_magnet:3d:mean_y']
    magno_mean_z=df_magnet['raw_magnet:3d:mean_z']

    (pitch,roll,yaw)=compute_roll_yaw_pitch(magno_mean_x,magno_mean_y,magno_mean_z)

    df_one['magno_pitch']=pitch
    df_one['magno_roll']=roll
    df_one['magno_yaw']=yaw

    y=df_one[['label:FIX_running','label:FIX_walking','label:SITTING','label:SLEEPING','label:OR_standing']]

    # to avoid null values
    y['label:FIX_running']=y['label:FIX_running'].fillna(0)
    y['label:FIX_walking']=y['label:FIX_walking'].fillna(0)
    y['label:SITTING']=y['label:SITTING'].fillna(0)
    y['label:SLEEPING']=y['label:SLEEPING'].fillna(0)
    y['label:OR_standing']=y['label:OR_standing'].fillna(0)

    #Check rows where all the recorded activities are zero ~ No activity recorded rows
    list_of_indexs=[]
    for i in range(len(y)):
        run=y.iloc[i,0]
        walk=y.iloc[i,1]
        sit=y.iloc[i,2]
        sleep=y.iloc[i,3]
        stand=y.iloc[i,4]
        activities=[run,walk,sit,sleep,stand]
        count_ones=activities.count(1)
        if walk==0 and run==0 and sit==0 and sleep==0 and stand==0:
            list_of_indexs.append(i)
        #check if more then 1 exists for different activities
        elif count_ones>1:
            list_of_indexs.append(i)

    y=y.drop(list_of_indexs)
    X=df_one.iloc[:,-9:]
    X=X.drop(list_of_indexs)


    return X,y

In [6]:
def load_model_1D(X_train):
    # Create sequential model 
    cnn_model = tf.keras.models.Sequential()
    #First CNN layer  with 32 filters, conv window 3, relu activation and same padding
    cnn_model.add(Conv1D(filters=32, kernel_size=(3,), padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.001), input_shape = (X_train.shape[1],1)))
    #Second CNN layer  with 64 filters, conv window 3, relu activation and same padding
    cnn_model.add(Conv1D(filters=64, kernel_size=(3,), padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
    #Third CNN layer with 128 filters, conv window 3, relu activation and same padding
    cnn_model.add(Conv1D(filters=128, kernel_size=(3,), padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
    #Fourth CNN layer with Max pooling
    cnn_model.add(MaxPool1D(pool_size=(3,), strides=2, padding='same'))
    cnn_model.add(Dropout(0.5))
    #Flatten the output
    cnn_model.add(Flatten())
    #Add a dense layer with 256 neurons
    cnn_model.add(Dense(units = 256, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
    #Add a dense layer with 512 neurons
    cnn_model.add(Dense(units = 512, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
    #Softmax as last layer with five outputs
    cnn_model.add(Dense(units = 5, activation='softmax'))    
    return cnn_model

In [7]:
# Load model and data (MobileNetV2, CIFAR-10)
#model = tf.keras.applications.MobileNetV2((32, 32, 3), classes=10, weights=None)
#model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])

#load data 
(X,y)=load_data()

#reversing the one-hot encoding
a=np.array(y)
y=np.where(a)[1]

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30)

#y_train=np.array(y_train).reshape(y_train.shape[0],y_train.shape[1],1)
#y_test=np.array(y_test).reshape(y_test.shape[0],y_test.shape[1],1)

model=load_model_1D(X_train)

#reshaping the arrays imply CNN
X_train = np.array(X_train).reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = np.array(X_test).reshape(X_test.shape[0], X_test.shape[1], 1)

model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [8]:
# Define Flower client
class CifarClient(fl.client.NumPyClient):
  def get_parameters(self, config):
    return model.get_weights()

  def fit(self, parameters, config):
    print(".... Fit Function Called ... ")
    model.set_weights(parameters)
    model.fit(X_train, y_train, epochs=60, batch_size=32)
    return model.get_weights(), len(X_train), {}

  def evaluate(self, parameters, config):
    print(".... Evaluate Function Called ... ")
    model.set_weights(parameters)
    loss, accuracy = model.evaluate(X_train, y_train)
    print("Testing accuracy is ",accuracy)
    return loss, len(X_train), {"accuracy": accuracy}

In [9]:
# # Start Flower client
# fl.client.start_numpy_client(server_address="127.0.0.1:8080", client=CifarClient())

In [10]:
# getting the model weights
orig_weights = model.get_weights()
print("Weights are ",orig_weights)

# these are the random weights

Weights are  [array([[[ 0.1039536 , -0.09282151, -0.19336729,  0.1703772 ,
          0.24193391, -0.20699611, -0.13277274, -0.10017613,
          0.2018193 ,  0.15910932, -0.16158047,  0.19994748,
         -0.03488848,  0.2351856 , -0.08226223,  0.17403194,
          0.06487495, -0.14658043, -0.1527815 , -0.18721932,
          0.12190986, -0.2364486 ,  0.09695148,  0.09003955,
          0.00239709, -0.09493193,  0.02289835,  0.16190377,
         -0.01589702,  0.00124779, -0.24182966, -0.04384169]],

       [[ 0.06811655,  0.19435507, -0.11293855,  0.11139196,
         -0.20063028,  0.19838023,  0.10859269, -0.00835495,
          0.19893494, -0.18989833, -0.11576283, -0.09419169,
         -0.04102829,  0.0199044 , -0.10122599, -0.16857457,
         -0.1919486 , -0.01499218, -0.00993136, -0.16558103,
         -0.10285194,  0.10991442,  0.1608415 , -0.02455935,
          0.06454551, -0.00081867,  0.13908255, -0.14532343,
          0.0992313 ,  0.00296278, -0.19354156, -0.01639739]],

    

In [11]:
# weight dimensions
# print(weights[5].shape)

In [12]:
# Training the model

# model.set_weights(weights)
# model.fit(X_train, y_train, epochs=60, batch_size=32)

In [13]:
# to visualize the weights, take mean difference of the weights and compare overtime

In [14]:
# calculating and plotting average of weights
temp = []
for i in range(0, len(orig_weights)):
    temp.append(np.mean(orig_weights[i]))

print(np.mean(temp))

0.001086675


In [15]:
# function to get mean value for a given set of parameters
def get_mean_param(params):
    temp = []
    for i in range(0, len(orig_weights)):
        temp.append(np.mean(orig_weights[i]))

    return (temp, np.mean(temp))
    # return temp

print(get_mean_param(orig_weights))

([0.01158575, 0.0, 0.000888416, 0.0, 0.00017141557, 0.0, 5.1969942e-05, 0.0, -1.9270448e-05, 0.0, 0.00036181865, 0.0], 0.001086675)


In [16]:
# now training for different weights and storing the parameters to see their behaviour
mean_params = []
def multi_train(initial_weights, max_epochs):
    model.set_weights(initial_weights)
    mean_params.append(get_mean_param(model.get_weights()))
    for i in range(10, max_epochs+1, 10):
        model.set_weights(initial_weights)
        model.fit(X_train, y_train, epochs=i, batch_size=32) 
        mean_params.append(get_mean_param(model.get_weights()))
        print(f"Training done for {i} epochs")

multi_train(orig_weights, 20)

Epoch 1/10
85/85 [==============================] - 7s 18ms/step - loss: 0.9550 - accuracy: 0.6379
Epoch 2/10
85/85 [==============================] - 1s 13ms/step - loss: 0.5630 - accuracy: 0.8258
Epoch 3/10
85/85 [==============================] - 1s 12ms/step - loss: 0.4846 - accuracy: 0.8512
Epoch 4/10
85/85 [==============================] - 1s 12ms/step - loss: 0.4388 - accuracy: 0.8645
Epoch 5/10
85/85 [==============================] - 1s 12ms/step - loss: 0.4251 - accuracy: 0.8670
Epoch 6/10
85/85 [==============================] - 1s 12ms/step - loss: 0.4337 - accuracy: 0.8604
Epoch 7/10
85/85 [==============================] - 1s 12ms/step - loss: 0.3945 - accuracy: 0.8740
Epoch 8/10
85/85 [==============================] - 1s 12ms/step - loss: 0.3725 - accuracy: 0.8781
Epoch 9/10
85/85 [==============================] - 1s 12ms/step - loss: 0.3609 - accuracy: 0.8803
Epoch 10/10
85/85 [==============================] - 1s 12ms/step - loss: 0.3770 - accuracy: 0.8726
Training 

In [17]:
print(mean_params[0])
print(mean_params[1])
print(mean_params[2])

([0.01158575, 0.0, 0.000888416, 0.0, 0.00017141557, 0.0, 5.1969942e-05, 0.0, -1.9270448e-05, 0.0, 0.00036181865, 0.0], 0.001086675)
([0.01158575, 0.0, 0.000888416, 0.0, 0.00017141557, 0.0, 5.1969942e-05, 0.0, -1.9270448e-05, 0.0, 0.00036181865, 0.0], 0.001086675)
([0.01158575, 0.0, 0.000888416, 0.0, 0.00017141557, 0.0, 5.1969942e-05, 0.0, -1.9270448e-05, 0.0, 0.00036181865, 0.0], 0.001086675)


In [18]:
print(model.get_weights())

[array([[[ 4.37341407e-02, -5.76055981e-02, -1.70099586e-01,
          1.63264632e-01,  2.94754624e-01, -2.25897461e-01,
         -1.82872280e-01, -9.94185135e-02,  1.79752320e-01,
          1.69371545e-01, -1.60288841e-01,  2.19158351e-01,
         -2.11938680e-03,  2.59909987e-01, -1.06982425e-01,
          1.40608221e-01,  9.77146253e-02, -1.30518138e-01,
         -1.45783857e-01, -2.83514440e-01,  1.43033400e-01,
         -1.95390865e-01,  8.69807005e-02,  1.20063588e-01,
          1.25345476e-02, -5.94290234e-02, -1.56061975e-02,
          1.81919843e-01,  8.45664740e-03,  2.67273709e-02,
         -2.57136762e-01, -5.16252257e-02]],

       [[ 6.41588867e-02,  1.85588658e-01, -8.61316472e-02,
          1.33136347e-01, -1.94752634e-01,  1.85428679e-01,
          9.88812968e-02, -2.73709204e-02,  1.86932683e-01,
         -1.70952573e-01, -1.07541770e-01, -9.06949937e-02,
         -1.43642556e-02,  2.71432158e-02, -6.86485618e-02,
         -1.65682748e-01, -2.04619825e-01, -2.7930552

In [19]:
print(orig_weights)

[array([[[ 0.1039536 , -0.09282151, -0.19336729,  0.1703772 ,
          0.24193391, -0.20699611, -0.13277274, -0.10017613,
          0.2018193 ,  0.15910932, -0.16158047,  0.19994748,
         -0.03488848,  0.2351856 , -0.08226223,  0.17403194,
          0.06487495, -0.14658043, -0.1527815 , -0.18721932,
          0.12190986, -0.2364486 ,  0.09695148,  0.09003955,
          0.00239709, -0.09493193,  0.02289835,  0.16190377,
         -0.01589702,  0.00124779, -0.24182966, -0.04384169]],

       [[ 0.06811655,  0.19435507, -0.11293855,  0.11139196,
         -0.20063028,  0.19838023,  0.10859269, -0.00835495,
          0.19893494, -0.18989833, -0.11576283, -0.09419169,
         -0.04102829,  0.0199044 , -0.10122599, -0.16857457,
         -0.1919486 , -0.01499218, -0.00993136, -0.16558103,
         -0.10285194,  0.10991442,  0.1608415 , -0.02455935,
          0.06454551, -0.00081867,  0.13908255, -0.14532343,
          0.0992313 ,  0.00296278, -0.19354156, -0.01639739]],

       [[-0.12504

In [20]:
# when we start with same initial parameters it converges. When given any other set it does not.
# However the mean remains same


In [21]:
# now training for different weights and storing the parameters to see their behaviour
mean_params = []
def multi_train(initial_weights, max_epochs):
    model.set_weights(initial_weights)
    mean_params.append(get_mean_param(model.get_weights()))
    for i in range(10, max_epochs+1, 10):
        # model.set_weights(initial_weights)
        model.fit(X_train, y_train, epochs=i, batch_size=32) 
        mean_params.append(get_mean_param(model.get_weights()))
        print(f"Training done for {i} epochs")

multi_train(orig_weights, 20)

Epoch 1/10
85/85 [==============================] - 1s 12ms/step - loss: 0.9032 - accuracy: 0.6696
Epoch 2/10
85/85 [==============================] - 1s 12ms/step - loss: 0.5288 - accuracy: 0.8387
Epoch 3/10
85/85 [==============================] - 1s 12ms/step - loss: 0.4590 - accuracy: 0.8604
Epoch 4/10
85/85 [==============================] - 1s 13ms/step - loss: 0.4265 - accuracy: 0.8674
Epoch 5/10
85/85 [==============================] - 1s 13ms/step - loss: 0.4047 - accuracy: 0.8755
Epoch 6/10
85/85 [==============================] - 1s 12ms/step - loss: 0.4047 - accuracy: 0.8703
Epoch 7/10
85/85 [==============================] - 1s 12ms/step - loss: 0.3985 - accuracy: 0.8722
Epoch 8/10
85/85 [==============================] - 1s 12ms/step - loss: 0.3782 - accuracy: 0.8755
Epoch 9/10
85/85 [==============================] - 1s 12ms/step - loss: 0.3567 - accuracy: 0.8792
Epoch 10/10
85/85 [==============================] - 1s 13ms/step - loss: 0.3742 - accuracy: 0.8744
Training 

In [22]:
print(mean_params[0])
print(mean_params[1])
print(mean_params[2])

([0.01158575, 0.0, 0.000888416, 0.0, 0.00017141557, 0.0, 5.1969942e-05, 0.0, -1.9270448e-05, 0.0, 0.00036181865, 0.0], 0.001086675)
([0.01158575, 0.0, 0.000888416, 0.0, 0.00017141557, 0.0, 5.1969942e-05, 0.0, -1.9270448e-05, 0.0, 0.00036181865, 0.0], 0.001086675)
([0.01158575, 0.0, 0.000888416, 0.0, 0.00017141557, 0.0, 5.1969942e-05, 0.0, -1.9270448e-05, 0.0, 0.00036181865, 0.0], 0.001086675)


In [23]:
print(model.get_weights())

[array([[[ 0.07349493, -0.08027287, -0.17845005,  0.19517788,
          0.27702647, -0.22322312, -0.16703595, -0.0894538 ,
          0.26312777,  0.16557615, -0.16860557,  0.19561046,
          0.00174898,  0.24841975, -0.11517366,  0.13965881,
          0.05510369, -0.11841843, -0.14414775, -0.34535408,
          0.09075373, -0.17269018,  0.09500001,  0.1001028 ,
         -0.00293325, -0.07403179,  0.01135305,  0.17890756,
          0.08854519,  0.02264339, -0.2951737 , -0.06037255]],

       [[ 0.09969909,  0.15516731, -0.09457954,  0.07464732,
         -0.2132574 ,  0.17705406,  0.11069948, -0.02111688,
          0.17285956, -0.15995036, -0.10349145, -0.0903343 ,
         -0.01705264,  0.04447031, -0.06653705, -0.13944133,
         -0.20850378, -0.02860341, -0.03016414, -0.17074129,
         -0.06228765,  0.05062522,  0.17748933, -0.02524253,
          0.07747297, -0.01444214,  0.1318562 , -0.1730522 ,
          0.0817586 , -0.0154865 , -0.21527755, -0.00901101]],

       [[-0.06645